<h1 style="color: blue; text-align: center;">Multi-task Cascaded Convolutional Networks (MTCNN) for Face Detection and Facial Landmark Alignment Using Parallel</h1>

---
## Members of Team:

| MSSV | Full Name | ID Github |
| ---- | ---- | -------- |
| 1612835 | Bùi Trọng Xuyến | btxuyenhcmus |
| 1612165 | Nguyễn Đào Vinh Hải | NDVHaiHCMUS |
| 1612859 | Nguyễn Đình Hữu | NguyenDinhHuu-HCMUS |

---
# 1.  Mô tả ứng dụng

MTCNN là viết tắt của Multi-Task Cascaded Convolutional Networks, bao gồm 3 mạng CNN (P-Net, R-Net, O-Net) xếp chồng lên nhau và cùng hoạt động phát hiện khuôn mặt. 

### Input:

Input của bài toán là một đoạn video hoặc hình ảnh.

### Output:

Output của bài toán là một đoạn video hoặc hình ảnh chứa vị trí (bounding box) của khuôn mặt và các vị trí trên khuôn mặt như mắt, mũi, miệng,...

![OutputMTCNN](Output.png)

<center>Ouput minh họa Bài Toán MTCNN</center> 

### Ứng dụng thực tiễn:

MTCNN có rất nhiều ứng dụng thực tiễn trong đời thực như là:
- Việc nhận diện khuôn mặt có rất nhiều lợi ích trong cuộc sống hiện tại của con người: nhận diện tội phạm, phát hiện vật, phát hiện vật thể lạ, thực hiện việc chấm công tự động, theo dõi tội phạm,...


### Lí do cần song song hóa bài toán:

- Việc song song hóa bài toán sẽ giúp chúng ta tiết kiệm được chi phí thời gian, sử dụng được tối đa tài nguyên phần cứng hiện tại.
- Giảm được độ trễ của bài toán.
- Tăng tốc độ training của bài toán.

### Lí do bài toán có thế song song hóa:

Do bài toán của MTCNN có sử dụng lớp convolution (tích chập ma trận).

![convolution](Convolution_schematic.gif "Convolution")

---
# 2. Cài đặt tuần tự

### B1: Ảnh đầu vào sẽ được resize theo nhiều kích cỡ khác nhau theo mô hình Image Pyramid

![Resize](Resize.png )

<center>Resize Image theo mô hình Image Pyramid (Link nguồn: https://medium.com/@iselagradilla94/multi-task-cascaded-convolutional-networks-mtcnn-for-face-detection-and-facial-landmark-alignment-7c21e8007923)</center> 


### B2: Sau khi Resize Image theo mô hình Image pyramid chúng ta sẽ đưa ảnh đã được resize vào một mạng fully convolutional network đó là Proposal-Network (viết tắt là P-Net). Chức năng của P-Net là phát hiện các windows có chứa mặt người. Ở bước này thì P-Net sẽ cho ra xác suất của một khuôn mặt nằm trong mỗi bounding boxes và xác suất của một khuôn mặt nằm trong mỗi bounding boxes

<img src="P-Net.jpeg" style= "width:750px;height:350px"/>
<center>Cấu trúc mô hình P-Net</center> 

### B3: Tiếp theo chúng ta sẽ sử dụng bounding box regression ở phần P-Net để làm input cho Refine-Network (R-Net). Kết quả cho ra của R-Net đó là tọa độ mới của các bounding boxes, cùng độ tin tưởng của nó.

<img src="R-Net.jpeg" style= "width:750px;height:350px"/>
<center>Cấu trúc mô hình R-Net</center> 

### B4: Cuối cùng là chúng ta sẽ sử dụng Output-Network (O-Net). Input của O-Net chính là các bounding boxes từ R-Net. Kết quả ở bước cuối cùng này là xác suất của khuôn mặt nằm trong bounding box, tọa độ của bounding box và tọa độ của các mốc trên khuôn mặt (vị trí mắt, mũi, miệng)

<img src="O-Net.jpeg" style= "width:750px;height:350px"/>
<center>Cấu trúc mô hình O-Net</center> 

![MTCNN](MTCNN.png)
<center>Hình ảnh minh họa cho cấu trúc làm việc của MTCNN</center>

### Để thực hiện được việc nhận dạng trong MTCNN chúng ta sẽ phải làm những tasks như sau:

#### Task 1: Face Classification

Vì đây là bài toán phân lớp chỉ có 2 class (face/non-face) nên chúng ta có thể tính được cross-entropy loss của từng mẫu $x_{i}$ theo công thức như sau:

${L_{i}}^{det} = -({y_{i}}^{det} log(p_{i}) + (1 - {y_{i}}^{det})(1 - log(p_{i})))$

Trong đó: $p_{i}$ là là xác suất gương mặt của của mẫu $x_{i}$. $y_{i} \in {0,1}$ là giá trị của ground-truth.

#### Task 2: Bounding box regression

Với mỗi windows dự đoán khuôn mặt chúng ta sẽ so sánh độ chênh lệch giữa windows đó với ground-truth của bài toán. Việc tính toán về độ chênh lệch đó được tính toán dựa trên cách tính khoảng cách Euclid đối với mỗi mẫu $x_i$, chúng ta  có công thức như sau:

${L_{i}}^{box} = {||{{\hat{y}}_i}^{box} - {y_{i}}^{box}||}_2^2$

Trong đó ${{\hat{y}}_i}^{box}$ là tọa độ bounding box của gương mặt lấy được từ việc train network, ${y_{i}}^{box}$ là tọa độ thực của bounding box của gương mặt từ ground-truth.

#### Task 3: Facial landmark localization:

Tương tự như bài toán Bounding box regression, chúng ta sẽ tính độ chênh lệch của facial landmark localization dựa việc tính toán khoảng cách Euclid như sau:

${L_{i}}^{landmark} = {||{{\hat{y}}_i}^{landmark} - {y_{i}}^{landmark}||}_2^2$

Trong đó ${{\hat{y}}_i}^{landmark}$ là tọa độ các đặc điểm của gương mặt lấy được từ việc train network, ${y_{i}}^{landmark}$ là tọa độ thực của các đặc điểm của gương mặt từ ground-truth.

#### Task 4: Multi-source traing:

---
# 3. Cài đặt song song (trên GPU)

---
# 4. Cài đặt song song (trên GPU) + tối ưu hóa

---
# 5. Nhìn lại quá trình làm đồ án

---
# 6. Tài liệu tham khảo